# PRAW Library
## Python Reddit API Wrapper

In [2]:
# install library
# !pip install praw

In [3]:
import praw
import pandas as pd
import networkx as nx
#import matplotlib.pyplot as plt

In [4]:
clientid = "EppA5GzbdocKzA"
clientsecret = "PBMQFMsqWuXuqGak1qVNlhzyi4A"
password = 'password'
username = "Aidenberg"
user_agent = "testcript by /u/aiden"

In [5]:
reddit = praw.Reddit(client_id=clientid, 
                     client_secret=clientsecret,
                     password= password, 
                     user_agent='testcript by /u/aiden',
                     username='Aidenberg')

In [6]:
reddit.user.me()

Redditor(name='Aidenberg')

### Play with PRAW

In [43]:
# load submissions title
subreddit = reddit.subreddit('python')
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    print(submission.title)

/r/Python official Job Board!
What's everyone working on this week?
How do I make an executable that doesn't require installing any python packages?
Jupyter Lab Release!
I created a quite useful django automated (database) logging library.


In [45]:
# load more information
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    print(submission.title,
          submission.ups,
          submission.downs,
          submission.visited)

/r/Python official Job Board! 128 0 False
What's everyone working on this week? 3 0 False
How do I make an executable that doesn't require installing any python packages? 60 0 False
Jupyter Lab Release! 161 0 False
I created a quite useful django automated (database) logging library. 12 0 False


In [42]:
# What else we can do with praw
subreddit.subscribe()
subreddit.unsubscribe()

In [47]:
# Comments
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    comments = submission.comments
    for comment in comments:
        print(20*'-')
        print(comment.body)

--------------------
Parent ID: 75jklc
Comment ID: doh5b96
Hi! I'm, uh, kinda desperate for work. I come from a poor family and they've run out of money to put me through college, and since I was just going there for a rubber stamp anyway I figure I can get away with just going straight on to work.

I've been working with Python for around eight years or so; don't even remember when I started learning to program. I've successfully delivered products to people a couple of times and I'm pretty comfortable with whatever technology you want me to use.

I *generally* prefer doing native application\backend stuff, and most of my experience is in tools, glue-code, and scripting. I'll do whatever you need me to do though, I'm happy as long as I can code. :P

Here's my GitLab: https://gitlab.com/toastengineer

Here's the most polished project I have going at the moment, although, uh, maybe don't look at the code for it *first*, I wrote most of it in three days for a game jam: http://schilcote.i

In [50]:
# Comments with threads
# error occurs if thread is too long.

#try again with 'news' subreddit
hot_python = reddit.subreddit('python').hot(limit=5)
for submission in hot_python:
    comments = submission.comments
    for comment in comments:
        print(20*'-')
        print('Parent ID:', comment.parent())
        print('Comment ID:', comment.id)
        print(comment.body)

--------------------
Parent ID: 82eye4
Comment ID: dv9kgox
"This is my own private domicile and I will not be harassed... bitch!"
--------------------
Parent ID: 82eye4
Comment ID: dv9mdqc
"No, ociffer, I'm not drunk driving... this is my home and I'm on my couch..."
--------------------
Parent ID: 82eye4
Comment ID: dv9ip2h
Been like this in Canada for years. Used to be a guy just a couple of hours outside of Moncton, lived in his car for years. Named Ricky I think. He argued the case himself, famous in Canadian law. ‘If I can’t smoke and swear I’m fucked’ - something like that. 
--------------------
Parent ID: 82eye4
Comment ID: dv9nebi
I just have this little trick where I think my skeleton is my body, and my skin is my home. No matter where I am, I’m always home
--------------------
Parent ID: 82eye4
Comment ID: dv9m30k
It won't just affect homeless people, but also poor and fugal people too. It's become an issue in San Francisco where instead of paying absurd prices to rent/own, p

AttributeError: 'MoreComments' object has no attribute 'parent'

In [51]:
hot_python = reddit.subreddit('news').hot(limit=1)
for submission in hot_python:
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        print(20*'-')
        print('Parent ID:', comment.parent())
        print('Comment ID:', comment.id)
        print(comment.body)

--------------------
Parent ID: 82eye4
Comment ID: dv9kgox
"This is my own private domicile and I will not be harassed... bitch!"
--------------------
Parent ID: 82eye4
Comment ID: dv9mdqc
"No, ociffer, I'm not drunk driving... this is my home and I'm on my couch..."
--------------------
Parent ID: 82eye4
Comment ID: dv9ip2h
Been like this in Canada for years. Used to be a guy just a couple of hours outside of Moncton, lived in his car for years. Named Ricky I think. He argued the case himself, famous in Canadian law. ‘If I can’t smoke and swear I’m fucked’ - something like that. 
--------------------
Parent ID: 82eye4
Comment ID: dv9nebi
I just have this little trick where I think my skeleton is my body, and my skin is my home. No matter where I am, I’m always home
--------------------
Parent ID: 82eye4
Comment ID: dv9m30k
It won't just affect homeless people, but also poor and fugal people too. It's become an issue in San Francisco where instead of paying absurd prices to rent/own, p

## Comment Network

In [8]:
col_edge = ['parentID', 'commentID']
lst = []
hot_python = reddit.subreddit('news').hot(limit=1)
for submission in hot_python:
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        lst.append([comment.parent().author, comment.author])
            
df_edge = pd.DataFrame(lst, columns=col_edge)

In [9]:
df_edge.head(5)

,parentID,commentID
0,cantstopthebald,Lefty_gun_nut
1,cantstopthebald,trepping
2,cantstopthebald,eddymarkwards
3,cantstopthebald,AkioDAccolade
4,cantstopthebald,hokaythxbai


In [10]:
col_node = ['userID']
df_node = pd.DataFrame(list(set(sum(lst, []))), columns=col_node)

In [11]:
G = nx.Graph()

for index, row in df_node.iterrows():
    G.add_node(row['userID'])
for index, row in df_edge.iterrows():
    G.add_edges_from([(row['parentID'], row['commentID'])])

In [12]:
#plt.figure(figsize=(25,25))
#options = {
#    'edge_color': '#FFDEA2',
#    'width': 1,
#    'with_labels': False,
#    'font_weight': 'regular',
#}
#
#pos = nx.kamada_kawai_layout(G)
#nx.draw(G, node_color = 'lightblue', node_size = [G.degree[i]*50 for i in G], pos=pos, **options)
#        
#ax = plt.gca()
#ax.collections[0].set_edgecolor("#555555") 
#plt.show()

<img src="comment_network.png">

### Collect node attributes

User Karma

User Subscription or active subreddits

Edge-wise attributes can be collected by considering common subreddits.


#### 1. Collect Karma

In [14]:
# Karma can be collected by redditor names
user = reddit.redditor("bboe")
user.comment_karma, user.link_karma

(8048, 2687)

In [15]:
# Karma can be collected by reddior ids (reddit.get function)
# Multiple requests

subreddit = reddit.subreddit('redditdev')
user_ids = [user.id for user in subreddit.moderator()]
data = reddit.get('api/user_data_by_account_ids',
                  params={'ids': ','.join(user_ids)})
for id_, d in data.items():
    print('{name}: {comment_karma} {link_karma}'.format(**d))

jedberg: 164261 46008
gooeyblob: 44452 9709
ketralnis: 49342 23878
KeyserSosa: 103014 50774
taylorkline: 1522 1495
bsimpson: 9196 17927
kn0thing: 195874 133274
chromakode: 42682 25347
spladug: 37494 15265
bboe: 8048 2687
kemitche: 50643 16541
spez: 613436 96433


#### 2. Collect Date the account created

In [16]:
# When were the accounts created?
import time
for id_, d in data.items():
    print('{}: {}'.format(d.get("name"),
                          time.ctime(d.get("created_utc"))))

jedberg: Thu Aug  4 00:00:00 2005
gooeyblob: Sat Nov 20 15:35:29 2010
ketralnis: Wed Nov 23 14:54:32 2005
KeyserSosa: Mon Jul 25 00:00:00 2005
taylorkline: Tue Dec 13 18:03:02 2016
bsimpson: Sun Jan  4 17:12:42 2009
kn0thing: Mon Jun  6 00:00:00 2005
chromakode: Sun Jun 11 21:29:04 2006
spladug: Thu Jun 18 12:43:00 2009
bboe: Fri Nov 13 13:04:01 2009
kemitche: Mon Jul 13 01:08:57 2009
spez: Mon Jun  6 00:00:00 2005


#### 4. Active subreddits per user

In [37]:
#user = praw.models.Redditor(reddit, name="/u/fwork", _data=None)
comments_path ="user/bboe/comments" 
comments = praw.models.ListingGenerator(reddit, comments_path, params=None, limit=100)
comments = comments.__iter__()

In [38]:
lst = []
for comment in comments:
    lst.append(comment.subreddit.display_name)

In [39]:
from collections import Counter
Counter(lst).most_common()

[('redditdev', 51),
 ('bitcoinpuzzles', 16),
 ('UCSantaBarbara', 12),
 ('SantaBarbara', 7),
 ('nyancoins', 2),
 ('Python', 1),
 ('RequestABot', 1),
 ('help', 1),
 ('bugs', 1),
 ('subreddit_stats', 1),
 ('tech', 1),
 ('teenagers', 1),
 ('garlicoin', 1),
 ('todayilearned', 1),
 ('learnpython', 1),
 ('changelog', 1),
 ('CrildeDevSandbox', 1)]

## Subreddit Network

In [20]:
## Creating a subreddit network
## connections among subreddits
## how to define edges?

In [21]:
subreddit = reddit.subreddit("programming")

In [22]:
#subreddit = reddit.subreddit('politics')
#   for submission in subreddit.submissions(1478592000, 1478678400):
#       print(submission.title)

##### Weight of edges can be assessed by common users

In [24]:
# Consider 7 different subreddits
subreddit_list = ["programming", "python", "technology", "coding", "django", "philosophy", "PhilosophyofReligion"]
LST = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    lst = []
    for i in subreddit.hot(limit=500):
        lst.append(i.author)
    LST.append(lst)

In [25]:
LST # returns Redditor object. Need to convert them into strings

[[Redditor(name='dzamir'),
  Redditor(name='Brainlag'),
  Redditor(name='shouldnotexist'),
  Redditor(name='aldacron'),
  Redditor(name='p-h'),
  Redditor(name='doom_Oo7'),
  Redditor(name='one_eyed_golfer'),
  Redditor(name='SpyR1014'),
  Redditor(name='alexeyr'),
  Redditor(name='wm32d'),
  Redditor(name='ljluzjvert'),
  Redditor(name='davidsiegel'),
  Redditor(name='magenta_placenta'),
  Redditor(name='IAmJohnnyHavana'),
  Redditor(name='psychologicalX'),
  Redditor(name='sunbas'),
  Redditor(name='Kok_Nikol'),
  Redditor(name='javinpaul'),
  Redditor(name='one_eyed_golfer'),
  Redditor(name='DeltaBurnt'),
  Redditor(name='michalg82'),
  Redditor(name='engberrg'),
  Redditor(name='neverforgetobama2'),
  Redditor(name='National_Blueberry'),
  Redditor(name='mariuz'),
  Redditor(name='one_eyed_golfer'),
  Redditor(name='one_eyed_golfer'),
  Redditor(name='sagatsnee'),
  Redditor(name='BigKev47'),
  Redditor(name='onidaito'),
  Redditor(name='viktoriia_shulga'),
  Redditor(name='henry_

In [26]:
# LST is a list of redditor objects
# Changing them into user names
l = []; L = []
for i in range(0,7):
    for j in LST[i]:
        if j != None:
            l.append(j.name)
    L.append(l)
    l = []

In [27]:
D = pd.DataFrame(columns = subreddit_list,
                 index = subreddit_list) # contingency matrix storing the number of common users

for i in range(0,7): # for loop for common users
    for j in range(0,7):
        if i < j:
            common_users = set(L[i]).intersection(L[j])
            print('Subreddits:({}, {}) \n Len:{} \n Users: {}'.format(subreddit_list[i], subreddit_list[j], 
                                                                      len(common_users), common_users))
            print(20*'-')
            
            D.at[subreddit_list[i],subreddit_list[j]] = D.at[subreddit_list[j],subreddit_list[i]] = len(common_users)

Subreddits:(programming, python) 
 Len:18 
 Users: {'Bogdanp', 'paulknysh', 'speckz', 'm_razali', 'oilshell', 'liranbh', 'botafoxy', 'blarghmatey', 'juanpabloaj', 'RubiksCodeNMZ', 'mariuz', 'psangrene', 'junp1289', 'The-unreliable-one', 'agumonkey', 'subbytech', 'grepawk', 'gerryjenkinslb'}
--------------------
Subreddits:(programming, technology) 
 Len:7 
 Users: {'iamcerberus', 'speckz', 'arkydon', 'svick', 'semi23', 'trot-trot', 'viktoriia_shulga'}
--------------------
Subreddits:(programming, coding) 
 Len:20 
 Users: {'paulknysh', 'speckz', 'ksyucs', 'sagatsnee', 'trushin', 'joebaf', 'artpar', '_patrickg', 'juanpabloaj', 'cdawg9089', 'javinpaul', 'aaronwhite47', 'yez', 'aboullaite', 'iamkeyur', 'minaandrawos', 'grepawk', 'engberrg', 'nfrankel', 'jmswlms'}
--------------------
Subreddits:(programming, django) 
 Len:0 
 Users: set()
--------------------
Subreddits:(programming, philosophy) 
 Len:2 
 Users: {'saranshk', 'iamcerberus'}
--------------------
Subreddits:(programming, Phi

In [28]:
D

,programming,python,technology,coding,django,philosophy,PhilosophyofReligion
programming,NaN,18,7,20,0,2,0
python,18,NaN,3,10,13,1,0
technology,7,3,NaN,1,0,4,0
coding,20,10,1,NaN,1,0,0
django,0,13,0,1,NaN,0,0
philosophy,2,1,4,0,0,NaN,7
PhilosophyofReligion,0,0,0,0,0,7,NaN


##### Edges by X-post
We may also use X-posts. X-post is cross-post meaning it locates in multiple subreddits.

In [30]:
subreddit_list = ["programming", "python", "technology", "coding", "django", "philosophy", "PhilosophyofReligion"]
LST = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    lst = []
    for i in subreddit.search("x-post", limit=500):
        lst.append(i.author)
    LST.append(lst)

In [52]:
import re #library for regular expression

LST2 = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    
    lst = []
    for i in subreddit.search("x-post", limit=500):
        tmp = re.search(r'r\/(.*?(?=[\]\)\s\/\:]))', i.title)
        if tmp != None:
            lst.append(tmp.group(1))
            
    LST2.append(lst)

In [53]:
LST2
# We can also make edge attributes of weights based on x-posts.

[['Games',
  'Software',
  'SQL',
  'javascript',
  'java',
  'Algorithms',
  'webdev',
  'learnpython',
  'redditdata',
  'MarioMaker',
  'Unix',
  'UniversityofReddit',
  'emulation',
  'TIL',
  'javascript',
  'compsci',
  'linux',
  'git',
  'Forth',
  'rails',
  'php',
  'javascript',
  'rust',
  'Seattle',
  'LLVM',
  'opensource',
  'Minecraft',
  'linux',
  'linux',
  'golang',
  'webdev',
  'rust',
  '$',
  'microsoft',
  'learnprogramming',
  'webdev',
  'interestingasfuck',
  'compsci',
  'webdev',
  'dcpu16',
  'emulation',
  'gamedev',
  'videos',
  'linux_programming',
  'python',
  'dataisbeautiful',
  'MachineLearning',
  'leagueoflegends',
  'nim',
  'morningcupofcoding',
  'Google',
  'rust',
  'learnprogramming',
  'space',
  'LunaLang',
  'c_programming',
  'apple',
  'rust',
  'compsci',
  'rust',
  'cplusplus',
  'rust',
  'Games',
  'rust',
  'linux',
  'rust',
  'InternetIsBeautiful',
  'firefox',
  'cpp',
  'webdev',
  'proceduralgeneration',
  'programmerchat'